In [1]:
import pandas as pd
import logging
import pickle
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
from sklearn.compose import ColumnTransformer
from datetime import datetime

# Configuración de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def load_data():
    """Cargar el dataset de vinos y devolverlo como DataFrame de pandas"""
    logger.info("Cargando el dataset de vinos...")
    data = load_wine()
    df = pd.DataFrame(data.data, columns=data.feature_names)
    df['target'] = data.target
    logger.info("Dataset de vinos cargado con éxito.")
    return df

def preprocess_data(df):
    """Limpiar y preparar datos para entrenamiento"""
    logger.info("Iniciando preprocesamiento de datos...")
    X = df.drop(columns=['target'])
    y = df['target']
    
    # Pipeline para preprocesamiento
    numerical_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),  # Completar valores faltantes con la media
        ('scaler', StandardScaler())  # Escalar variables numéricas
    ])
    
    # Composición final del preprocesador
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_pipeline, X.columns)
        ])
    
    logger.info("Preprocesamiento completo.")
    return preprocessor, X, y

def train_model(preprocessor, X, y):
    """Entrenar el modelo de clasificación"""
    logger.info("Dividiendo los datos en conjunto de entrenamiento y prueba...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Pipeline final de entrenamiento
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', DecisionTreeClassifier(random_state=42))
    ])
    
    # Entrenar el modelo
    logger.info("Entrenando el modelo...")
    pipeline.fit(X_train, y_train)
    logger.info("Entrenamiento completado.")
    
    # Evaluación del modelo
    logger.info("Evaluando el modelo...")
    y_pred = pipeline.predict(X_test)
    logger.info("\n" + classification_report(y_test, y_pred))
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    logger.info(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}')
    
    return pipeline

def save_model(model, filename):
    """Guardar el modelo entrenado en un archivo .pkl"""
    logger.info(f"Guardando el modelo en {filename}...")
    with open(filename, 'wb') as file:
        pickle.dump(model, file)
    logger.info("Modelo guardado con éxito.")

def main():
    logger.info("Inicio del pipeline de clasificación de vinos.")
    df = load_data()
    preprocessor, X, y = preprocess_data(df)
    model = train_model(preprocessor, X, y)
    save_model(model, f'model_wine_{datetime.now().strftime("%Y%m%d%H%M")}.pkl')
    logger.info("Pipeline completado con éxito.")

if __name__ == "__main__":
    main()


2024-11-01 15:42:25,334 - INFO - Inicio del pipeline de clasificación de vinos.
2024-11-01 15:42:25,336 - INFO - Cargando el dataset de vinos...
2024-11-01 15:42:25,361 - INFO - Dataset de vinos cargado con éxito.
2024-11-01 15:42:25,362 - INFO - Iniciando preprocesamiento de datos...
2024-11-01 15:42:25,363 - INFO - Preprocesamiento completo.
2024-11-01 15:42:25,364 - INFO - Dividiendo los datos en conjunto de entrenamiento y prueba...
2024-11-01 15:42:25,365 - INFO - Entrenando el modelo...
2024-11-01 15:42:25,372 - INFO - Entrenamiento completado.
2024-11-01 15:42:25,373 - INFO - Evaluando el modelo...
2024-11-01 15:42:25,379 - INFO - 
              precision    recall  f1-score   support

           0       0.93      0.93      0.93        14
           1       0.93      1.00      0.97        14
           2       1.00      0.88      0.93         8

    accuracy                           0.94        36
   macro avg       0.95      0.93      0.94        36
weighted avg       0.95    